In [ ]:
import snowflake.snowpark as snowpark
from snowflake.snowpark.context import get_active_session

# --- Configuration ---
NUM_USERS = 1  # Set to 1 for testing
USER_PREFIX = "user"
USER_PASSWORD_PREFIX = "SLMholuser"
# ---------------------

# Get active session
session = get_active_session()

print("--- Starting Full Lab Setup ---")

try:
    # Set context to ACCOUNTADMIN for the entire setup
    session.use_role("ACCOUNTADMIN")
    session.use_database("DATAOPS_EVENT_PROD")
    session.use_schema("PUBLIC")
    print("Initial context set to ACCOUNTADMIN, DATAOPS_EVENT_PROD.PUBLIC.")

    # ======================================================================
    # Part 1: Create Shared Resources (Integrations & Parent Role)
    # ======================================================================

    session.sql("CREATE DATABASE IF NOT EXISTS snowflake_intelligence").collect()
    session.sql("CREATE SCHEMA IF NOT EXISTS snowflake_intelligence.agents").collect()
    
    print("\n--- 1. Creating PARENT role for shared permissions ---")
    session.sql("CREATE ROLE IF NOT EXISTS LAB_PARENT_ROLE").collect()
    session.sql("GRANT ROLE LAB_PARENT_ROLE TO ROLE SYSADMIN").collect()
    print("✅ Created LAB_PARENT_ROLE.")

    print("\n--- 2. Creating Shared Integrations ---")
    session.sql("""
    CREATE OR REPLACE NETWORK RULE Snowflake_intelligence_WebAccessRule
      MODE = EGRESS
      TYPE = HOST_PORT
      VALUE_LIST = ('0.0.0.0:80', '0.0.0.0:443')
    """).collect()

    session.sql("""
    CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION Snowflake_intelligence_ExternalAccess_Integration
      ALLOWED_NETWORK_RULES = (Snowflake_intelligence_WebAccessRule)
      ENABLED = true
    """).collect()
    print("✅ Network Rule and External Access Integration created.")

    print("\n--- 3. Granting Shared Permissions to PARENT Role ---")
    session.sql("GRANT USAGE ON INTEGRATION Snowflake_intelligence_ExternalAccess_Integration TO ROLE LAB_PARENT_ROLE").collect()
    session.sql("GRANT DATABASE ROLE SNOWFLAKE.PYPI_REPOSITORY_USER TO ROLE LAB_PARENT_ROLE").collect()
    print("✅ Shared integration and PyPI permissions granted to LAB_PARENT_ROLE.")

    # ======================================================================
    # Part 2: Create Individual User Environments
    # ======================================================================
    
    print(f"\n--- Starting provisioning loop for {NUM_USERS} user(s) ---")

    for i in range(NUM_USERS):
        # Define all user-specific names
        username = f"{USER_PREFIX}{i}"
        password = f"{USER_PASSWORD_PREFIX}{i}"
        user_role_name = f"ATTENDEE_ROLE_USER{i}"
        user_db_name = f"CORTEX_LAB_DB_USER{i}"
        user_schema_name = f"RISK_DATA_USER{i}"
        user_wh_name = f"LAB_WH_USER{i}"
        
        print(f"\nProcessing {username}...")
        
        # 0. Ensure we are ACCOUNTADMIN
        session.use_role("ACCOUNTADMIN")
        
        # 1. Create Personal Role
        session.sql(f'CREATE ROLE IF NOT EXISTS "{user_role_name}"').collect()
        
        # 2. Create User
        print(f"Creating user {username}, with role: {user_role_name}")
        session.sql(f"""
            CREATE USER IF NOT EXISTS {username} 
            PASSWORD='{password}' 
            DEFAULT_ROLE="{user_role_name}" 
            MUST_CHANGE_PASSWORD=FALSE
        """).collect()

        print(f"Granting user: {username} role: {user_role_name}")
        # 3. Grant Role to User and Parent Role to User's Role
        session.sql(f'GRANT ROLE "{user_role_name}" TO USER {username}').collect()
        print(f"Grainting user: {username} role: LAB_PARENT_ROLE ")
        session.sql(f'GRANT ROLE LAB_PARENT_ROLE TO ROLE "{user_role_name}"').collect()
        
        # 4. Create Personal Warehouse
        print(f"Creating warehouse: {user_wh_name}")
        session.sql(f"""
            CREATE OR REPLACE WAREHOUSE "{user_wh_name}" 
            WAREHOUSE_SIZE = XSMALL 
            AUTO_SUSPEND = 60
        """).collect()
        print("after warehouse creation")
        
        # 5. Create Personal Database and Schema
        print(f"Creating db: {user_db_name}")
        session.sql(f'CREATE OR REPLACE DATABASE "{user_db_name}" DATA_RETENTION_TIME_IN_DAYS = 0').collect()
        session.use_database(f'"{user_db_name}"')
        print(f"Creating schema: {user_schema_name}")
        session.sql(f'CREATE OR REPLACE SCHEMA "{user_schema_name}"').collect()
        
        # Grant OWNERSHIP privileges
        print("Granting ALL PRIVILEGES to the new role...")
        print(f"details: {user_wh_name}, {user_role_name}")
        session.sql(f'GRANT OWNERSHIP ON WAREHOUSE "{user_wh_name}" TO ROLE "{user_role_name}"').collect()
        session.sql(f'GRANT OWNERSHIP ON DATABASE "{user_db_name}" TO ROLE "{user_role_name}"').collect()
        session.sql(f'GRANT OWNERSHIP ON SCHEMA "{user_db_name}"."{user_schema_name}" TO ROLE "{user_role_name}"').collect()
        print("Granting Snowflake Intelligence access")
        session.sql(f'GRANT USAGE ON DATABASE SNOWFLAKE_INTELLIGENCE TO ROLE "{user_role_name}"').collect()
        session.sql(f'GRANT USAGE ON SCHEMA SNOWFLAKE_INTELLIGENCE.AGENTS TO ROLE "{user_role_name}"').collect()
        session.sql(f'GRANT CREATE AGENT ON SCHEMA SNOWFLAKE_INTELLIGENCE.AGENTS TO ROLE "{user_role_name}"').collect()
        
        print(f"✅ Successfully created user, role, DB, schema, and WH for {username}.")

    # Reset session context to the original one
    session.use_database("DATAOPS_EVENT_PROD")
    session.use_schema("PUBLIC")
    
    final_status = f"🎉 FULL LAB SETUP COMPLETE for {NUM_USERS} user(s)."
    print(f"\n--- {final_status} ---")

except Exception as e:
    print(f"\n❌ ERROR during setup: {e}")
    final_status = f"❌ ERROR: {e}"

# Return the final status to the notebook cell output
final_status

In [ ]:
import snowflake.snowpark as snowpark
from snowflake.snowpark.context import get_active_session

session = get_active_session()

print("--- 1. Checking for 'MANAGE GRANTS' on ACCOUNTADMIN ---")
session.use_role("ACCOUNTADMIN")
session.use_warehouse("DEFAULT_WH")

try:
    # 1. Show privileges granted TO the ACCOUNTADMIN role.
    session.sql("SHOW GRANTS TO ROLE ACCOUNTADMIN").collect()
    
    # 2. Query the output of the SHOW command.
    sql = """
    SELECT "privilege", "granted_on", "name", "grantee_name"
    FROM TABLE(RESULT_SCAN(LAST_QUERY_ID()))
    WHERE "privilege" = 'MANAGE GRANTS' AND "granted_on" = 'ACCOUNT';
    """
    
    df = session.sql(sql).to_pandas()
    
    if df.empty:
        print("\n❌ VALIDATION FAILED:")
        print("Your ACCOUNTADMIN role does NOT have the 'MANAGE GRANTS' privilege.")
        print("This is likely why you're getting 'insufficient privileges' on GRANT commands.")
    else:
        print("\n✅ VALIDATION PASSED:")
        print("Your ACCOUNTADMIN role HAS 'MANAGE GRANTS' privilege.")
        print(df)

except Exception as e:
    print(f"Error checking grants: {e}")


In [ ]:
import snowflake.snowpark as snowpark
from snowflake.snowpark.context import get_active_session

session = get_active_session()

print("\n--- 2. Checking for 'CREATE' privileges on ACCOUNTADMIN ---")
session.use_role("ACCOUNTADMIN")

try:
    # This command lists ALL privileges the role has
    session.sql("SHOW GRANTS OF ROLE ACCOUNTADMIN").collect()
    
    # This query scans the results of the command above
    sql = """
    SELECT "privilege", "granted_on"
    FROM TABLE(RESULT_SCAN(LAST_QUERY_ID())) 
    WHERE "privilege" IN (
        'CREATE DATABASE', 
        'CREATE SCHEMA', 
        'CREATE WAREHOUSE', 
        'CREATE ROLE', 
        'CREATE USER'
    )
    AND "granted_on" = 'ACCOUNT';
    """
    
    df = session.sql(sql).to_pandas()
    
    print(f"\nFound {len(df)} 'CREATE' privileges:")
    print(df)
    
    if len(df) < 5:
        print("\nWARNING: Your ACCOUNTADMIN is also missing some 'CREATE' privileges.")
    else:
        print("\n✅ This confirms your role can create objects.")

except Exception as e:
    print(f"Error checking grants: {e}")